In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

from typing import List
from tqdm import trange

import wandb
from wandb.keras import WandbCallback

import config, music_model, utils
from sklearn.utils import class_weight

### CONFIGURATION ###

wandb.login()

ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
MODEL_SAVE_PATH = os.path.join(ROOT_PATH, 'training', 'checkpoints', 'genre_classifier')
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
MODEL_SAVE_PATH = os.path.join(MODEL_SAVE_PATH, 'model-{val_loss:.2f}.h5')
DATASET_NAME = 'lmd_matched_final_2048_cut'
USE_SMALL_GENRE_SET = DATASET_NAME == 'tf_data7dict'
USE_ONE_GPU = True

WEIGHTS_PATH = os.path.join(ROOT_PATH, 'training', 'checkpoints', 
                            'model_GPT_baseline_with_mse_vellmd_matched_2048', 
                            'model_GPT_baseline_with_mse_vellmd_matched_2048')

conf = config.Config("single_instruments_type", ROOT_PATH)

if USE_SMALL_GENRE_SET:
    conf.accepted_subgenres = ['folk', 'nes', 'maestro']
# If we need to use only the first GPU
if USE_ONE_GPU:
    conf.GPUS = tf.config.experimental.list_physical_devices('GPU')[0]
    conf.BATCH_SIZE = conf.BATCH_SIZE
    conf.GLOBAL_BATCH_SIZE = conf.BATCH_SIZE
    conf.num_devices = 1

### MODEL CREATION ###

if conf.num_devices > 1:
    print("Using multiple GPUs with Mirrored Strategy")
    with conf.training_strategy.scope():
        model = music_model.create_model(conf,
                                         num_genres=len(conf.accepted_subgenres),
                                         use_regularization=False,
                                         use_masking_layers=False)
else:
    print("Using single GPU/CPU device")
    model = music_model.create_model(conf,
                                     num_genres=len(conf.accepted_subgenres),
                                     use_regularization=False,
                                     use_masking_layers=False)

print("Loading pre-trained weights into the model and freezing weights...")
model.load_weights(WEIGHTS_PATH)
    
print("Collecting dataset splits...")
dataset_path = conf.dataset_paths[DATASET_NAME]
train_dataset, val_dataset, test_dataset = utils.get_dataset_splits(dataset_path, conf)

print("Modifying labels...")
train_dataset, val_dataset, test_dataset = \
    train_dataset.map(lambda x, y: (x, x[1])), \
    val_dataset.map(lambda x, y: (x, x[1])),   \
    test_dataset.map(lambda x, y: (x, x[1]))

2023-05-15 11:11:02.185410: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 11:11:02.324914: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-15 11:11:02.364142: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-15 11:11:03.005144: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Using single GPU/CPU device


2023-05-15 11:11:09.597628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 11:11:10.246291: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30970 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


Loading pre-trained weights into the model and freezing weights...
Modifying labels...


We extract the transformer's embeddings for the dataset and define the genre classifier on top of the model

In [3]:
music_transformer = tf.keras.Model(
    inputs=model.inputs, 
    outputs=model.get_layer('tfgpt2_model').output.last_hidden_state
)
music_transformer.trainable = False

class_gen = tf.keras.Sequential([
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(conf.accepted_subgenres), activation='relu'),
    tf.keras.layers.Softmax()
])

out_classes = class_gen(music_transformer.outputs[0])

genre_classifier = tf.keras.Model(inputs=music_transformer.inputs, outputs=out_classes)
genre_classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                         loss=tf.keras.losses.KLDivergence())

In [4]:
labels = np.concatenate([x[1] for x, y in train_dataset], axis=0)
genre_labels = np.where(labels > 0)[1]
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(genre_labels), y=genre_labels)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

In [ ]:
run = wandb.init(project="Music Generation", entity="marcello-e-federico",
                         group='genre_classification', job_type='train',
                         name='baseline_genre_class_full')

genre_classifier.fit(x=train_dataset, validation_data=val_dataset, epochs=200, 
                     callbacks=[
                        tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True),
                        tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_SAVE_PATH, 
                            save_best_only=True, save_weights_only=True),
                        tf.keras.callbacks.ReduceLROnPlateau(),
                        WandbCallback(save_model=False, 
                                      save_graph=False,log_weights=False)
                        ])

run.finish()

wandb: Currently logged in as: volpepe (marcello-e-federico). Use `wandb login --relogin` to force relogin


Epoch 1/200


2023-05-15 11:11:37.493745: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


5314/6949 [=====================>........] - ETA: 5:21 - loss: 1.5757

In [ ]:
results = genre_classifier.evaluate(x=test_dataset, batch_size=conf.BATCH_SIZE)

In [ ]:
results